## Syntactical shortcut - Separate code with line breaks

If you have lines that are long and hard to read, putting in line breaks can help. In Python, you can have line breaks inside parentheses. Let's demonstrate this on a piece of code we've written yesterday:

In [3]:
# import data from yesterday
import pandas as pd
gapminder = pd.read_table("gapminderDataFiveYear_superDirty.txt", sep = "\t")
gapminder['region'] = gapminder['region'].astype(str)
gapminder_copy = gapminder.copy()

# Method 1 for formatting the 'region' column:
gapminder_copy['region'] = gapminder_copy['region'].str.lstrip() # Strip white space on left
gapminder_copy['region'] = gapminder_copy['region'].str.rstrip() # Strip white space on right
gapminder_copy['region'] = gapminder_copy['region'].str.lower() # Convert to lowercase

# Method 2 for formatting the 'region' column:
gapminder_copy['region'] = gapminder_copy['region'].str.lstrip().str.rstrip().str.lower() # Strip white space on left and right, and convert to lowercase
                            
print(gapminder_copy['region'])

0       asia_afghanistan
1       asia_afghanistan
2       asia_afghanistan
3       asia_afghanistan
4       asia_afghanistan
              ...       
1715     africa_zimbabwe
1716     africa_zimbabwe
1717     africa_zimbabwe
1718     africa_zimbabwe
1719     africa_zimbabwe
Name: region, Length: 1720, dtype: object


There are three different transformations happening above: removing whitespace on the left, removing whitespace on the right, and converting the text to lowercase. We can make this one line more intuitive by breaking it up into three:

In [6]:
# New method of chaining functions
gapminder_copy['region'] = (
    gapminder_copy['region']
    .str.lstrip()
    .str.rstrip()
    .str.lower()
)
#makes very clear what order of operations is & how functions interplay
print(gapminder_copy['region'])

0       asia_afghanistan
1       asia_afghanistan
2       asia_afghanistan
3       asia_afghanistan
4       asia_afghanistan
              ...       
1715     africa_zimbabwe
1716     africa_zimbabwe
1717     africa_zimbabwe
1718     africa_zimbabwe
1719     africa_zimbabwe
Name: region, Length: 1720, dtype: object


We get the same output as above! This code is functionally the same as methods 1 and 2. We benefit from explicitly delineating each step like in method 1, and we also get the nicer syntax of applying all cleaning steps at the same time with method 2.

## Outlining your code

Thinking about what you want your future code to do for you *before* coding anything reduces the time you spend physically coding. It forces you to think about the big pieces that go into solving your problem and how they'll fit together, revealing potential problems much earlier. Let's take an example from day 1 to illustrate:

In [ ]:
percent = 20
if percent < 38:
    print('Low')
elif percent < 47:
    print('Normal')
else:
    print('High')

To make this code more relevant to biology, let's introduce a real biological variable: *hematocrit*. Hematocrit is the volume percentage of red blood cells in blood. The normal values for humans are:
* Males: 41% - 50%
* Females: 36% - 44%
* Average: 39% - 47%  -  these numbers used in the code above

Now let's think about our code:
1. We first check if the percent is less than 38: if so, then label as "Low"
2. We then check if the percent is less than 47: if so, then label as "Normal"
3. Otherwise, label as "High"

This code seems intuitive from a first glance, but there's a conceptual oversight: the second case here isn't actually correct. Values less than 47% are normal *only if* the value isn't already less than 38%. The logic presented in this code works by virtue of checking the 38% case before the 47% case. Suppose we unintentionally coded in case 2 before case 1. Now we first check for values less than 47, and any values that fulfill that condition are "Normal", even if they're *also* less than 38. This is an easy accident that can lead to erroneous results.

Let's think a bit more about the biological meaning of HCT percentage: it's more accurate to explicitly state the values that fall under each category:
* Values between 0% - 38% are "Low"
* Values between 39% - 47% are "Normal"
* Values between 48% - 100% are "High"

To follow this biological meaning, we would rewrite our code as follows:
1. Check if the percent is between 0 and 38: if so, then label as "Low"
2. Then, check if the percent is between 39 and 47: if so, then label as "Normal"
3. Then, check if the percent is between 48 and 100: if so, then label as "High"
4. **(new condition!)** If it's any other integer, raise an error

This is better! We've done a few defensive programming concepts here:
* Written up **pseudo-code**: this is not actual code, but an outline of how you want your code to be structured
* Sanitized our input and guarded against a potential error
* More explicitly stated the biological meaning of our code
* Defended against the concept of a "wrong order" for our if/else statements - now it doesn't matter how the three conditions are ordered

So now this code would look like:

In [14]:
# New code goes here
percent = 20
if 0 <= percent <= 38:
    print('Low')
elif 38 <= percent <= 47:
    print('Normal')
elif 47 <= percent <= 100:
    print('High')
else:
    raise ValueError(f"The percent input {percent} is out of range (0-100)")

Low


# Working with files

When working with real-world data, it will typically be in a file, and **not** in your code. Fortunately, Python has functions to read files. These work with simple text files, and if you need to handle images or other binary formats, there are libraries that can help with that.

Sequence data is typically in stored text files, like fasta. So let's walk through reading one of those.

## Open and close

When you use a word processor or spreadsheet, you open files, work with them, and then close them when you're done. In Python, you do the same thing.

In [15]:
f = open('ae.fa')
for line in f:
    print(line)
f.close()

>lcl|AE014075.1_gene_2 [locus_tag=c0002] [location=534..911]

GTGTTCTACAGAGAGAAGCGTAGAGCAATAGGCTGTATTTTGAGAAAGCTGTGTGAGTGGAAAAGTGTAC

GGATTCTGGAAGCTGAATGCTGTGCAGATCATATCCATATGCTTGTGGAGATCCCGCCCAAAATGAGCGT

ATCAGGCTTTATGGGATATCTGAAAGGGAAAAGCAGTCTGATGCCTTACGAGCAGTTTGGTGATTTGAAA

TTCAAATACAGGAACAGGGAGTTCTGGTGCAGAGGGTATTACGTCGATACGGTGGGTAAGAACACGGCGA





Let's go through the steps we just did.
1. We used the `open()` *function* on a string that represents a path to a file.
    * The result of that function was saved to the variable `f`. This value is called a *file object*.
2. We wrote a for loop. When you write a for loop for a file object, each loop variable represents a line in the file.
3. We printed the loop variable for each loop.
4. We used the *method* `.close()` to close the file.

## Reading lines

When we work with text files, we can read them line by line in a loop. Each line of text from the file is set into our loop variable, and we print it out from the loop.

You'll probably notice that we have a blank line in between each line from the file. Text files and programs indicate that there is a new line using a special newline character, `\n`. In our previous example, each line in the file includes the `\n` newline character at the end.

Let's look into this by adding each line to a single string, all_lines, and compare printing the string vs the raw data.

In [24]:
f = open('ae.fa')
all_lines = ''
for line in f:
    all_lines = all_lines + line
f.close()
print(all_lines) # First output is the print result
all_lines # Second is what the string data looks like

>lcl|AE014075.1_gene_2 [locus_tag=c0002] [location=534..911]
GTGTTCTACAGAGAGAAGCGTAGAGCAATAGGCTGTATTTTGAGAAAGCTGTGTGAGTGGAAAAGTGTAC
GGATTCTGGAAGCTGAATGCTGTGCAGATCATATCCATATGCTTGTGGAGATCCCGCCCAAAATGAGCGT
ATCAGGCTTTATGGGATATCTGAAAGGGAAAAGCAGTCTGATGCCTTACGAGCAGTTTGGTGATTTGAAA
TTCAAATACAGGAACAGGGAGTTCTGGTGCAGAGGGTATTACGTCGATACGGTGGGTAAGAACACGGCGA




'>lcl|AE014075.1_gene_2 [locus_tag=c0002] [location=534..911]\nGTGTTCTACAGAGAGAAGCGTAGAGCAATAGGCTGTATTTTGAGAAAGCTGTGTGAGTGGAAAAGTGTAC\nGGATTCTGGAAGCTGAATGCTGTGCAGATCATATCCATATGCTTGTGGAGATCCCGCCCAAAATGAGCGT\nATCAGGCTTTATGGGATATCTGAAAGGGAAAAGCAGTCTGATGCCTTACGAGCAGTTTGGTGATTTGAAA\nTTCAAATACAGGAACAGGGAGTTCTGGTGCAGAGGGTATTACGTCGATACGGTGGGTAAGAACACGGCGA\n\n'

If we know we have the whole line, we can strip off the newline character with the `.strip()` method.

In [59]:
string_with_newline = "There is a newline at the end\n"
string_with_newline

'There is a newline at the end\n'

In [60]:
string_with_newline.strip()

'There is a newline at the end'

Let's try this with our original code to print each line in a file.

In [61]:
f = open('ae.fa')
for line in f:
    line = line.strip()
    print(line, end='')
f.close()

>lcl|AE014075.1_gene_2 [locus_tag=c0002] [location=534..911]GTGTTCTACAGAGAGAAGCGTAGAGCAATAGGCTGTATTTTGAGAAAGCTGTGTGAGTGGAAAAGTGTACGGATTCTGGAAGCTGAATGCTGTGCAGATCATATCCATATGCTTGTGGAGATCCCGCCCAAAATGAGCGTATCAGGCTTTATGGGATATCTGAAAGGGAAAAGCAGTCTGATGCCTTACGAGCAGTTTGGTGATTTGAAATTCAAATACAGGAACAGGGAGTTCTGGTGCAGAGGGTATTACGTCGATACGGTGGGTAAGAACACGGCGA

## Reading a fasta file

Write a function called `read_fasta(filename)` that takes in the input filename and returns a single string for all of the DNA sequences in the file.

In [65]:
#Your code here!
def read_fasta(filename):
    f = open(filename)
    lines = ''
    for line in f:
        if line.startswith(">"):
            continue
        lines = lines + line.strip()
    f.close()
    return(lines)
read_fasta("ls_orchid.fasta")

'...'

In [66]:
print(read_fasta('ae.fa'))

GTGTTCTACAGAGAGAAGCGTAGAGCAATAGGCTGTATTTTGAGAAAGCTGTGTGAGTGGAAAAGTGTACGGATTCTGGAAGCTGAATGCTGTGCAGATCATATCCATATGCTTGTGGAGATCCCGCCCAAAATGAGCGTATCAGGCTTTATGGGATATCTGAAAGGGAAAAGCAGTCTGATGCCTTACGAGCAGTTTGGTGATTTGAAATTCAAATACAGGAACAGGGAGTTCTGGTGCAGAGGGTATTACGTCGATACGGTGGGTAAGAACACGGCGA


In [67]:
def reverse_complement(dna_sequence):
    """Reverses the complement of a dna sequence"""
    complements = {"T":"A", "A":"T", "C":"G", "G":"C"}
    reverse = dna_sequence[::-1]
    result = ""
    for letter in reverse:
        result = result + complements[letter]
    return(result)

print(reverse_complement(read_fasta('ae.fa')))

TCGCCGTGTTCTTACCCACCGTATCGACGTAATACCCTCTGCACCAGAACTCCCTGTTCCTGTATTTGAATTTCAAATCACCAAACTGCTCGTAAGGCATCAGACTGCTTTTCCCTTTCAGATATCCCATAAAGCCTGATACGCTCATTTTGGGCGGGATCTCCACAAGCATATGGATATGATCTGCACAGCATTCAGCTTCCAGAATCCGTACACTTTTCCACTCACACAGCTTTCTCAAAATACAGCCTATTGCTCTACGCTTCTCTCTGTAGAACAC


## Scripts

We've done a good job of organizing our code into functions here, but we've only been running them from this notebook. So next, we're going to take our code and put it in a script - starting with the `read_fasta` function.

Let's start with a script that reads the ae.fa file specifically and prints it.

Notice that the first line contains a `%%` operator followed by the command writefile and a file name. This operator is specific to jupyter notebooks, called a "Cell Magic Command", and copies the code written in a cell into a file.

In [71]:
%%writefile read_fasta_v1.py
def read_fasta(filename): #this is wild!!!!
    """read a fasta file and returns all sequences concatenated"""
    sequence = ''
    f = open(filename)
    for line in f:
        line = line.strip()
        if ">" not in line:
            #append to last sequence
            sequence = sequence + line
    f.close()
    return(sequence)

print(read_fasta("ae.fa"))

Writing read_fasta_v1.py


Our script reads our `ae.fa` file every time we run it, but we know most programs don't work that way. The programs we used in bash expected a data file as an argument, and that's a good convention for programs we write too.

In Python, our program can get these arguments, but we have to load a module called `sys` from the standard library, a collection of modules included in python but not available by default. The documentation for these is part of the documentation for python: https://docs.python.org/3/library/sys.html

Libraries are incredibly useful - there are libraries for working with numeric and scientific data, generating plots, fetching data from the web, working with image and document files, databases, etc. And of course, there's a library for getting things like your script's command-line arguments.

So, let's change our `read_fasta.py` program slightly.

In [73]:
%%writefile read_fasta_v2.py
import sys

def read_fasta(filename):
    """read a fasta file and returns all sequences concatenated"""
    sequence = ''
    f = open(filename)
    for line in f:
        line = line.strip()
        if ">" not in line:
            #append to last sequence
            sequence = sequence + line
    f.close()
    return sequence

print(read_fasta(sys.argv[1])) 
"""
says 'the second input to the script line when called is going inside this'.
sys.argv returns a list where the first item sys.argv[0] is the name of the script by default, 
and each additional item in the list are the command line arguments. 
If no argument was passed, sys.argv should be a list of just the script name
"""

Overwriting read_fasta_v2.py


But what happens if we don't have an input file name? According to the documentation, `sys.argv` returns a list where the first item `sys.argv[0]` is the name of the script by default, and each additional item in the list are the command line arguments. If no argument was passed, `sys.argv` should be a list of just the script name.

In [76]:
%%writefile read_fasta_v3.py
import sys

def read_fasta(filename):
    """read a fasta file and returns all sequences concatenated"""
    sequence = ''
    f = open(filename)
    for line in f:
        line = line.strip()
        if ">" not in line:
            #append to last sequence
            sequence = sequence + line
    f.close()
    return sequence

#want to be able to import this later, but not immediately run:
if __name__ == "__main__": #if you're running this file directly and NOT importing, it will be run
#then continue. this needs to be between the def and the data/run statements.
    if len(sys.argv) < 2:
        print('Usage:', sys.argv[0], '<sequence.fa>')
        sys.exit(1)

    print(read_fasta(sys.argv[1])) 

Overwriting read_fasta_v3.py


## Making scripts you can import

So far, we have used modules to help us work on our analyses such as:
- Standard Library
    - sys
- Third Party
    - pandas
    - numpy
    - matplotlib

These are imported using the `import` keyword and we can use functions from them. We also write functions for use in our own code. Having these available to import into other scripts gives the benefit of:
1. Letting us reuse code over multiple analyses (DRY)
2. Letting others use our code in their own scripts without copy/pasting (DRY)

While it may seem like going out of one's way to write a module and a script for analysis, you can actually have one python file act as both a module and run it from the command line to perform a task.

In your Jupyter notebook server (in your browser):
* New -> Python3 Notebook
    * name: **demo_for_imports**

In **demo_for_imports.ipynb**:

In [ ]:
import read_fasta_v3

But, why does it print some output when we import into our jupyter notebook?

This is because we have a print() statement at the bottom of cool_functions.py! This is not good practice. How do we separate the print statement from the useful functions in our .py document???

Answer:
Add this weird statement `if __name__ == "__main__":` before the data & print statement (after the function, still in our `automation_python_2022.ipynb` document):

### Exercise: Extending your Python scripts to do more

In your `demo_for_imports.ipynb` notebook, you've now imported `read_fasta_v3`, which returns a single DNA string from a fasta file. Building off of this module, how would you get a count of unique dinucleotides ("AA, AT, AC, AG, TA, etc.) for each fasta file? Writing pseudocode may help if you're not sure how to approach it.

In [ ]:
# Put your answer here!



## The next level of automation: combining Python and bash

Suppose you're given a few *hundred* fasta files you need to concatenate. You could type them all into a list in your Jupyter notebook and run it, or you could have bash automate the Python script for you!

First, a technical check: in your Git Bash or Terminal window, run `python --version`. Hopefully, you get some info about the version of python you're running.

Recall our bash lesson on the first day:

Using the `python` command in the terminal, we can also run Python files without needing to open up Jupyter notebook! You use this `python` command just like any other bash command:

You should get a lot of letters outputted to your window. Better yet, let's redirect it to another file to save for later:

You may have seen the `>` bash symbol before, which means to redirect the output of a command into another file. Here, we use `>>`: it's is similar in that it also redirects output to a file, but this *concatenates* the result to whatever is in the file, instead of overwriting the whole file with the new results.

Now we can open up and check `output_fastas.txt`. There's one line for every file processed.

Okay, time for one more level of automation: it's great that we can get bash to loop over our Python file and our input files, but what if we don't want to type out the `for` loop in bash every single time we run an analysis? We can store that command in a `bash` script, similar to how we stored our Python code in a Python script.

In either `nano` or `vim`, copy and paste the above code into a new file called `script.sh`. Remember that we can create a new file by directly typing `nano script.sh` or `vim.sh` on the command line.

Once you've created your `script.sh` file, run it on the command line with:

It should work as intended (i.e. not output anything to the terminal), and you can open up your `output_fastas.txt` file to see the results.

### Exercise - optimizing your script

How would you modify this `script.sh` so that it empties the contents of `output_fastas.txt` before running your program? Hint: there are multiple bash commands you could potentially use.

## We can go even further: walking away from your computer

Congrats, you have a bash script that automates a Python file over hundreds of fasta files! What if you were dealing with gigabytes (or even terabytes) of data? You'd probably be waiting forever for your script to finish, but you probably don't want to sit around that long. What can you do?

* `nohup` - Stands for `no hangup` - even if you close your bash terminal, the program will continue to run in the background of your computer. Just make sure you don't shut down your computer before it finishes! On a compute cluster, this isn't really a problem since compute clusters generally stay online 24/7.
* `&` - Run this program in the background of your terminal. This frees up your terminal so you can work on other things and run more commands. Note that this *does not* keep it running if the terminal is closed; you'll still need `nohup` for that.